In [1]:
from thermotools.plot import *

import netCDF4 as nc
import time, glob, os
import numpy as np

from thermotools import moles
from thermotools import get_data, empty_dir

fbig = 1e90
today = str(time.strftime("%Y-%m-%d"))
print(today)

2024-10-29


In [2]:
datdir = os.path.join(get_data(), "compiled", "dat")
empty_dir(datdir)

pltdir = os.path.join(get_data(), "compiled", "plt")
empty_dir(pltdir)

In [3]:
gases = ['H','C','N','O','S','Fe','Mg','Si','Na',
         'O2', 'O3', 'N2', 'N2O', 'NO', 'CH3', 'H2O2', 
         'SO2', 'NH3', 'H2', 'CO2', 'H2O', 'CO', 'HCl', 
         'HCN', 'CH4', 'H2SO4', 'H2S', 'OH', 'SiO', 'VO',
         'SiO2', 'FeO2', 'SO', 'S2', 'S8', 'OCS', 'TiO',
         'FeH', 'C2H4', 'C2H6', 'HNO3', 'Fe', 'PH3',
         'FeO','Na2','NaO','Mg','Mg2','MgO', 'CH3']

gases = list(set(gases))

In [4]:
elem_table = moles.read_elements()

In [5]:
# Find the name JANAF gives 'gas', based on number of atoms
def find_janaf_name(gas:str):

    target = moles.count_atoms(gas)

    files = glob.glob(os.path.join(get_data(),"cp","dat")+"/*.csv")
    for f in files:
        form  = moles.formula_from_path(f)
        atoms = moles.count_atoms(form)

        good = True  

        for k in atoms.keys():
            if k not in target.keys():
                good = False 
                break 

        for k in target.keys():
            if k not in atoms.keys():
                good = False 
                break  
            if target[k] != atoms[k]:
                good = False 
                break
        
        if good:
            return form 

    return ""


In [6]:
def compile(gas:str):
    print("Compiling %s"%gas)

    # JANAF alias 
    janaf = find_janaf_name(gas) 
    if len(janaf) == 0:
        print("    skipping (no JANAF data)")
        return 
    
    # MMW calculation
    mmw_val = moles.mmw_from_formula(gas, elem_table)
    
    # create dataset
    fpath = datdir+"/%s.nc"%gas
    if os.path.exists(fpath):
        os.remove(fpath)
    ds = nc.Dataset(fpath, 'w')

    # JANAF name 
    ds.createDimension('njanaf', len(janaf))
    NC_janaf = ds.createVariable('JANAF', 'S1', ('njanaf'))
    NC_janaf[:] = nc.stringtochar(np.array([janaf], 'S'))

    # Date created
    ds.createDimension('ndate', len(today))
    NC_today = ds.createVariable('created', 'S1', ('ndate'))
    NC_today[:] = nc.stringtochar(np.array([today], 'S'))

    # MMW
    NC_mmw = ds.createVariable("mmw","f8")
    NC_mmw[:] = mmw_val
    NC_mmw.units = "kg mol-1"

    # Load saturation data
    T_trip = 0.0
    T_crit = 1e9
    X_sat = [[0,5000],[fbig,fbig]]  # dummy values

    satdat = os.path.join(get_data(),"sat","dat")
    sat_path = satdat+"/%s_sat.csv"%gas
    if os.path.exists(sat_path):
        T_trip = float(np.loadtxt(satdat+"/%s_trip.csv"%gas))
        T_crit = float(np.loadtxt(satdat+"/%s_crit.csv"%gas))
        X_sat  = np.loadtxt(sat_path, delimiter=',').T
    else:
        print("    without saturation data")

    # Critical point
    NC_tcrit = ds.createVariable("T_crit","f8")
    NC_tcrit[:] = T_crit
    NC_tcrit.units = "K"

    # Triple point
    NC_ttrip = ds.createVariable("T_trip","f8")
    NC_ttrip[:] = T_trip
    NC_ttrip.units = "K"

    # Saturation curve 
    ds.createDimension("sat",len(X_sat[0]))
    #    temperatures
    NC_tsat = ds.createVariable("sat_T","f8","sat")
    NC_tsat[:] = X_sat[0][:]
    NC_tsat.units = "K"
    #    pressures
    NC_psat = ds.createVariable("sat_P","f8","sat")
    NC_psat[:] = X_sat[1][:]
    NC_psat.units = "Pa"
    
    # Latent heat 
    X_lat = [[0,5000],[0.0,0.0]]      # dummy values
    lat_path = os.path.join(get_data(),"lv","dat")+"/%s.csv"%gas
    if os.path.exists(lat_path):
        X_lat = np.loadtxt(lat_path, delimiter=',').T
    else:
        print("    without enthalpy data")
    ds.createDimension("lat",len(X_lat[0]))
    #    lookup temperatures 
    NC_dHt = ds.createVariable("lat_T","f8","lat")
    NC_dHt[:] = X_lat[0][:]
    NC_dHt.units = "K"
    #    enthalpy changes [J/kg]
    NC_dH = ds.createVariable("lat_H","f8","lat")
    NC_dH[:] = X_lat[1][:]
    NC_dH.units = "J kg-1"

    # Heat capacity
    cap_path = os.path.join(get_data(),"cp","dat")+"/%s.csv"%janaf
    X_cap = np.loadtxt(cap_path, delimiter=',').T
    ds.createDimension("cap",len(X_cap[0]))
    #    lookup temperatures 
    NC_cpt = ds.createVariable("cap_T","f8","cap")
    NC_cpt[:] = X_cap[0][:]
    NC_cpt.units = "K"
    #   heat capacity
    NC_cp = ds.createVariable("cap_C","f8","cap")
    NC_cp[:] = X_cap[1][:] / mmw_val # convert from "per mol" to "per kg"
    NC_cp.units = "J K-1 kg-1"

    # done 
    ds.close()
    print("    ok")


In [7]:
for g in gases:
    compile(g)

Compiling SiO2
    without saturation data
    without enthalpy data
    ok
Compiling Na2
    skipping (no JANAF data)
Compiling SO2
    ok
Compiling H2SO4
    without saturation data
    without enthalpy data
    ok
Compiling O2
    ok
Compiling SO
    without saturation data
    without enthalpy data
    ok
Compiling SiO
    without saturation data
    without enthalpy data
    ok
Compiling FeH
    skipping (no JANAF data)
Compiling C2H6
    skipping (no JANAF data)
Compiling H
    without saturation data
    without enthalpy data
    ok
Compiling MgO
    without saturation data
    without enthalpy data
    ok
Compiling FeO
    without saturation data
    without enthalpy data
    ok
Compiling HNO3
    without saturation data
    without enthalpy data
    ok
Compiling FeO2
    skipping (no JANAF data)
Compiling H2
    ok
Compiling NH3
    ok
Compiling OCS
    without saturation data
    without enthalpy data
    ok
Compiling S
    without saturation data
    without enthalpy data
  

In [8]:
# Make plots 

files = glob.glob(datdir+"/*.nc")

for f in files:

    gas = moles.formula_from_path(f)

    ds = nc.Dataset(f,'r')

    fig,axs = plt.subplots(3,1, sharex=True, figsize=(5,4))

    axs[0].set_title(gas)

    axs[0].plot(ds.variables["sat_T"][:],    ds.variables["sat_P"][:]*1e-5)
    axs[0].set_ylabel(r"P$_{\text{sat}}$ [bar]")

    axs[1].plot(ds.variables["lat_T"][:],    ds.variables["lat_H"][:]*1e-3)
    axs[1].set_ylabel(r"$\Delta$H [kJ kg$^{-1}$]")

    axs[2].plot(ds.variables["cap_T"][:],    ds.variables["cap_C"][:]*1e-3)
    axs[2].set_ylabel("Cp [kJ K$^{-1}$ kg$^{-1}$]")

    axs[-1].set_xlabel("Temperature [K]")
    fig.savefig(pltdir + "/%s.png"%gas, bbox_inches='tight', dpi=200)
    plt.close("all")

    ds.close()
